In [ ]:
import pandas as pd
import re
# from nltk.corpus import words
from nltk.tokenize import word_tokenize
import nltk
import numpy as np
#nltk.download('punkt')
from textblob import Word
pd.options.display.width = 0

import spacy
nlp = spacy.load('en_core_web_sm')
all_stopwords = nlp.Defaults.stop_words

pd.set_option('display.max_rows', 6000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
#pd.describe_option('display')
# import enchant

In [ ]:
def find_in_english_dict(text):
    for w in word_tokenize(text):
        if Word(w.strip().lower()).spellcheck()[0][1]!=1.0 and len(re.findall(r"[A-Z]{2,}", w))>0:
          # print(w)
          return True
    return False

find_in_english_dict('HYPERTHROIDSM')
Word('allergy'.strip().lower()).spellcheck()

In [ ]:
l = {'text_id':['A1','A2','A3','A4'], 'text':['a','b','c', 'd'], 'cui_l':[[1,2,5],[2,5], [],[]],
    'cui_g':[[1,2,3],[2,3],[4],[]], 'cui_d':[[1,2,4],[3], [], []]}
df = pd.DataFrame(data=l)
df['cui_len']=df['cui_l'].apply(lambda x: len(x))
df['cui_g_l']=(df['cui_g']+df['cui_d']).apply(lambda x: list(set(x)))
df['ID'] = ['ID'+ str(x) for x in df.index.tolist()]
df['cui_final'] = np.where(df['cui_len']!=0, df['cui_l'], df['cui_g_l'])
df_id_cui = df[['ID','cui_final']]
df, df_id_cui.explode('cui_final', ignore_index=True), 

In [ ]:
# merging new text translation
"""path='/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/'
path_output = '/nfs/home/vyasa/projects/proj_off/data_off/clarify/spanish_comorbidity/new/output/'

df = pd.read_csv(path+'comorbidities_modified_witht_abbr_new.csv', sep=',', error_bad_lines=True, encoding='utf-8')
df.shape

df_new_text = pd.read_csv(path_output+'nohaving_cuiMEV_done.csv', encoding='utf-8')
display(df_new_text.head(5))

df.loc[df['Value_ID'].isin(df_new_text.Value_ID),'replacement'] = df_new_text['replacement '].values
display(df[df['Value_ID'].isin(df_new_text.Value_ID)].head(10))
display (df[['Value_ID', 'replacement']].head(10))
df.to_csv(path+'comorbidities_modified_witht_abbr_new_new.csv', encoding='utf-8', index=False)"""

In [2]:
import scispacy
import spacy
from scispacy.umls_linking import UmlsEntityLinker

nlp = spacy.load("en_core_sci_lg")
linker = UmlsEntityLinker(resolve_abbreviations=True)
nlp.add_pipe(linker)
#nlp.add_pipe("scispacy_linker")

doc = nlp("Surgery Appendicitis")
print (doc.ents)
for entity in doc.ents:
	for umls_ent in entity._.umls_ents:
		print(linker.umls.cui_to_entity[umls_ent[0]])

(Surgery, Appendicitis)
CUI: C1274039, Name: General surgery specialty
Definition: A specialty in which manual or operative procedures are used in the treatment of disease, injuries, or deformities.
TUI(s): T091
Aliases: (total: 10): 
	 Surgery, General Surgery, General Surgery, surgery general, Surgery.general, general surgery, General surgery, General surgery, Surgery, General, General surgery (qualifier value)
CUI: C0038894, Name: Surgery specialty
Definition: The branch of medical science that treats disease or injury by operative procedures.
TUI(s): T091
Aliases: (total: 5): 
	 Surgery, Surgery, Surgery, Surgery, Surgery
CUI: C4084778, Name: Surgery - Clinical Class
Definition: None
TUI(s): T170
Aliases: (total: 2): 
	 Surgery, SURG
CUI: C1705196, Name: Surgery Domain
Definition: A subject domain utilized for the submission of information encompassing and representing data, vocabulary or records related to surgery.
TUI(s): T170
Aliases: (total: 3): 
	 Surgery Domain, Surgery, SG
C

In [3]:
import spacy
import scispacy

from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_lg")
nlp.add_pipe("scispacy_linker", config={"resolve_abbreviations": True, "linker_name": "umls"})

doc = nlp("Surgery Appendicitis")

# Each entity is linked to UMLS with a score
# (currently just char-3gram matching).
linker = nlp.get_pipe("scispacy_linker")
for entity in doc.ents:
    for umls_ent in entity._.kb_ents:
        print(linker.kb.cui_to_entity[umls_ent[0]])